## Capstone Project


#### Import necessary Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  46.97 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.14 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  37.30 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  48.56 MB/s
Libraries imported.


#### Define Foursquare Credentials and Version

In [3]:
CLIENT_ID =   # your Foursquare ID
CLIENT_SECRET =   # your Foursquare Secret
VERSION = '20190101'

#### Search for businesses around the pier in Hermosa Beach California

In [4]:
address = 'Hermosa Beach Pier, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 33.8615954, -118.4033929.


In [5]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API
radius = 1600 # define radius - 1600 meters ~= 1 mile

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL 

'https://api.foursquare.com/v2/venues/explore?&client_id=QDNSA5AF0V4GHNCOVLLT0LDWXBKUIWZIDG0MW2UPDVOXK1HJ&client_secret=LFWAWILOU43MYDKEO2QK5AMWUZXDZAPJJ35G11XY4JKHEYRE&v=20190101&ll=33.8615954,-118.4033929&radius=1600&limit=1000'

In [6]:
#Send the GET Request and examine the results
results = requests.get(url).json()


In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Hermosa Beach - The Strand,Beach,33.861906,-118.401690
1,Brother's Burritos,Burrito Place,33.861181,-118.401000
2,Palmilla Cocina Y Tequila,Spanish Restaurant,33.862099,-118.400985
3,Paciugo Gelato,Ice Cream Shop,33.861251,-118.399696
4,Good Stuff Restaurant,American Restaurant,33.862656,-118.401811


In [9]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


#### Determine the top venues in HB

In [10]:
grouped = nearby_venues.groupby('categories').count()
grouped.sort_values(by='name', ascending=False).head()

,name,lat,lng
categories,,,
Mexican Restaurant,7,7,7
Beach,6,6,6
Seafood Restaurant,5,5,5
Gym / Fitness Center,5,5,5
American Restaurant,4,4,4


#### Search for a coffee related venues

In [11]:
search_query = 'Coffee, Cafe'

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

coffee_results = requests.get(url).json()

# assign relevant part of JSON to venues
coffee_venues = coffee_results['response']['venues']

# tranform venues into a dataframe
coffee_df = json_normalize(coffee_venues)
coffee_df.shape


(27, 24)

#### Define information and interest and filter dataframe

In [12]:
# keep only columns that include venue name, and anything that is associated with location
coffee_filtered_columns = ['name', 'categories'] + [col for col in coffee_df.columns if col.startswith('location.')] + ['id']
coffee_df_filtered = coffee_df.loc[:, coffee_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
coffee_df_filtered['categories'] = coffee_df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
coffee_df_filtered.columns = [column.split('.')[-1] for column in coffee_df_filtered.columns]

coffee_df_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Coffee Bean & Tea Leaf,Coffee Shop,1133 Artesia Blvd,US,Manhattan Beach,United States,at Pacific Coast Highway,1548,"[1133 Artesia Blvd (at Pacific Coast Highway),...","[{'lng': -118.39473713253174, 'lat': 33.873500...",33.873500,-118.394737,90266,CA,4b8013d5f964a5200d5030e3
1,Planet Earth Eco Cafe,Café,509 Pier Ave,US,Hermosa Beach,United States,at Bard St,701,"[509 Pier Ave (at Bard St), Hermosa Beach, CA ...","[{'lng': -118.39668169366543, 'lat': 33.864536...",33.864537,-118.396682,90254,CA,4aee15fef964a520f2d121e3
2,Starbucks,Coffee Shop,1303 Hermosa Avenue,US,Hermosa Beach,United States,NaN,330,"[1303 Hermosa Avenue, Hermosa Beach, CA 90254,...","[{'lng': -118.40033636566274, 'lat': 33.863130...",33.863130,-118.400336,90254,CA,4b04880df964a520ec5422e3
3,The Source Cafe,Vegetarian / Vegan Restaurant,509 Pier Ave,US,Hermosa Beach,United States,Bard St,701,"[509 Pier Ave (Bard St), Hermosa Beach, CA 902...","[{'lng': -118.39665127938063, 'lat': 33.864498...",33.864499,-118.396651,90254,CA,50dca3a9e4b08766e1785d63
4,Cafe Bonaparte,Dessert Shop,NaN,US,Hermosa Beach,United States,NaN,248,"[Hermosa Beach, CA 90254, United States]","[{'lng': -118.40078111191218, 'lat': 33.862123...",33.862123,-118.400781,90254,CA,4f06585e4901d41f1d1d7c4b
5,The Lighthouse Café,Music Venue,30 Pier Ave,US,Hermosa Beach,United States,Strand,221,"[30 Pier Ave (Strand), Hermosa Beach, CA 90254...","[{'lng': -118.40103786268571, 'lat': 33.861967...",33.861967,-118.401038,90254,CA,453a22f8f964a520e33b1fe3
6,The Gum Tree Cafe & Boutique,Australian Restaurant,238 Pier Ave,US,Hermosa Beach,United States,NaN,474,"[238 Pier Ave, Hermosa Beach, CA 90254, United...","[{'lng': -118.39868036260397, 'lat': 33.863280...",33.863280,-118.398680,90254,CA,4b9c0361f964a520c64036e3
7,Starbucks,Coffee Shop,1100 Pacific Coast Hwy,US,Hermosa Beach,United States,at Aviation Blvd,1058,"[1100 Pacific Coast Hwy (at Aviation Blvd), He...","[{'lng': -118.39215175755803, 'lat': 33.863404...",33.863404,-118.392152,90254,CA,4b2e8f42f964a52079e224e3
8,Fritto Misto Italian Cafe,Italian Restaurant,316 Pier Ave,US,Hermosa Beach,United States,NaN,542,"[316 Pier Ave, Hermosa Beach, CA 90254, United...","[{'lng': -118.39814914483836, 'lat': 33.863774...",33.863774,-118.398149,90254,CA,4b1b2056f964a52084f823e3
9,Downtown Bakery Cafe,Food,37 Pier Ave,US,Hermosa Beach,United States,NaN,222,"[37 Pier Ave, Hermosa Beach, CA 90254, United ...","[{'lng': -118.401054, 'lat': 33.86206, 'label'...",33.862060,-118.401054,90254,CA,4f43b2c519834bc91f5854ca


#### Review data, filter out records where categories are not related to places where people would grab coffee

In [13]:
#exclude non-coffee related venues
categories = ['Coffee Shop']
coffee_shop_df_filtered = coffee_df_filtered[coffee_df_filtered.categories.isin(categories)]
coffee_shop_df_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Coffee Bean & Tea Leaf,Coffee Shop,1133 Artesia Blvd,US,Manhattan Beach,United States,at Pacific Coast Highway,1548,"[1133 Artesia Blvd (at Pacific Coast Highway),...","[{'lng': -118.39473713253174, 'lat': 33.873500...",33.873500,-118.394737,90266,CA,4b8013d5f964a5200d5030e3
2,Starbucks,Coffee Shop,1303 Hermosa Avenue,US,Hermosa Beach,United States,NaN,330,"[1303 Hermosa Avenue, Hermosa Beach, CA 90254,...","[{'lng': -118.40033636566274, 'lat': 33.863130...",33.863130,-118.400336,90254,CA,4b04880df964a520ec5422e3
7,Starbucks,Coffee Shop,1100 Pacific Coast Hwy,US,Hermosa Beach,United States,at Aviation Blvd,1058,"[1100 Pacific Coast Hwy (at Aviation Blvd), He...","[{'lng': -118.39215175755803, 'lat': 33.863404...",33.863404,-118.392152,90254,CA,4b2e8f42f964a52079e224e3
17,Billy Bobs Coffee House,Coffee Shop,NaN,US,Hermosa Beach,United States,NaN,1173,"[Hermosa Beach, CA 90254, United States]","[{'lng': -118.39087677001953, 'lat': 33.863346...",33.863346,-118.390877,90254,CA,4f0c5a82e4b0298d453df59f
22,Pursue Coffee,Coffee Shop,1503 Aviation Blvd,US,Redondo Beach,United States,NaN,2001,"[1503 Aviation Blvd, Redondo Beach, CA 90278, ...","[{'lng': -118.383233, 'lat': 33.868151, 'label...",33.868151,-118.383233,90278,CA,5b249bd351950e002ce6e732
23,Starbucks Coffee,Coffee Shop,1200 N Sepulveda Blvd,US,Manhattan Beach,United States,NaN,1568,"[1200 N Sepulveda Blvd, Manhattan Beach, CA 90...","[{'lng': -118.38663403265737, 'lat': 33.863795...",33.863796,-118.386634,90266,CA,4e4dc4d5bd41b76bef92d266


In [14]:
#exclude non-coffee related venues
other_categories = ['Café','Food','Breakfast Spot']
other_coffee_df_filtered = coffee_df_filtered[coffee_df_filtered.categories.isin(other_categories)]
other_coffee_df_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
1,Planet Earth Eco Cafe,Café,509 Pier Ave,US,Hermosa Beach,United States,at Bard St,701,"[509 Pier Ave (at Bard St), Hermosa Beach, CA ...","[{'lng': -118.39668169366543, 'lat': 33.864536...",33.864537,-118.396682,90254,CA,4aee15fef964a520f2d121e3
9,Downtown Bakery Cafe,Food,37 Pier Ave,US,Hermosa Beach,United States,NaN,222,"[37 Pier Ave, Hermosa Beach, CA 90254, United ...","[{'lng': -118.401054, 'lat': 33.86206, 'label'...",33.862060,-118.401054,90254,CA,4f43b2c519834bc91f5854ca
10,Bienvenue Au Cafe Bonaparte,Café,53 Pier Ave,US,Hermosa Beach,United States,NaN,285,"[53 Pier Ave, Hermosa Beach, CA 90254, United ...","[{'lng': -118.40031, 'lat': 33.86139, 'label':...",33.861390,-118.400310,90254,CA,59b93f53446ea65e09dcadcf
11,Bienvenue Auto Cafe Bonaparte,Café,53 Pier Ave,US,Hermosa Beach,United States,NaN,289,"[53 Pier Ave, Hermosa Beach, CA 90254, United ...","[{'lng': -118.40027, 'lat': 33.861362, 'label'...",33.861362,-118.400270,90254,CA,59b93ef8bfc6d071799a80b1
12,Back Burner Cafe,Food,87 14th St,US,Hermosa Beach,United States,NaN,324,"[87 14th St, Hermosa Beach, CA 90254, United S...","[{'lng': -118.400937, 'lat': 33.863679, 'label...",33.863679,-118.400937,90254,CA,4f3246f619836c91c7c7cdd3
13,Sara Beach Cafe,Café,87 14th St,US,Hermosa Beach,United States,NaN,343,"[87 14th St, Hermosa Beach, CA 90254, United S...","[{'lng': -118.40072112, 'lat': 33.863743, 'lab...",33.863743,-118.400721,90254,CA,4de29bf018385df2b0282d21
14,Uncle Stavros Cafe,Breakfast Spot,NaN,US,Hermosa Beach,United States,NaN,875,"[Hermosa Beach, CA 90254, United States]","[{'lng': -118.39830092870703, 'lat': 33.854968...",33.854968,-118.398301,90254,CA,50157fc5e4b0cfd78b797f15
15,Goulash cafe,Café,Palm Dr,US,Hermosa Beach,United States,NaN,562,"[Palm Dr, Hermosa Beach, CA, United States]","[{'lng': -118.400289, 'lat': 33.865939, 'label...",33.865939,-118.400289,NaN,CA,568e1102498e8c239b178b33
18,Roman Aroma Cafe,Café,901 North Catalina Ave,US,Redondo Beach,United States,NaN,1630,"[901 North Catalina Ave, Redondo Beach, CA 902...","[{'lng': -118.39152398640418, 'lat': 33.850761...",33.850761,-118.391524,90277,CA,56f98bea498e1d1fe0a39d26
19,Jitters Cafe,Food,190 Hermosa Ave,US,Hermosa Beach,United States,NaN,904,"[190 Hermosa Ave, Hermosa Beach, CA 90254, Uni...","[{'lng': -118.398006, 'lat': 33.854808, 'label...",33.854808,-118.398006,90254,CA,4f32165819836c91c7b47d79


In [15]:
other_coffee_df_filtered.groupby('categories').count()

,name,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
categories,,,,,,,,,,,,,,
Breakfast Spot,1,0,1,1,1,0,1,1,1,1,1,1,1,1
Café,6,6,6,6,6,1,6,6,6,6,6,5,6,6
Food,4,4,4,4,4,0,4,4,4,4,4,4,4,4


#### Let's visualize the Coffee Shops and Cafes that are nearby

In [16]:
coffee_venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Hermosa Beach Pier

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hermosa Beach Pier',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(coffee_venues_map)

# add the coffee shops and cafes as blue circle markers
for lat, lng, label in zip(coffee_shop_df_filtered.lat, coffee_shop_df_filtered.lng, coffee_shop_df_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(coffee_venues_map)
    
# add the coffee related venues (cafes, breakfast, etc) as orange circle markers
for lat, lng, label in zip(other_coffee_df_filtered.lat, other_coffee_df_filtered.lng, other_coffee_df_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        popup=label,
        fill = True,
        fill_color='orange',
        fill_opacity=0.6
    ).add_to(coffee_venues_map)    
    

#display map
coffee_venues_map

#### Load retail lease data from loopnet.com 

In [17]:
#Available Retail Leases in the HB area as of February 10, 2019
HB_Rental_Data = pd.DataFrame({
'Address': ['1200 Pacific Coast Hwy Hermosa Beach, CA 90254','1402 Pacific Coast Hwy Hermosa Beach, CA 90254','1310 Pacific Coast Hwy Hermosa Beach, CA 90254','1880 Pacific Coast Hwy Hermosa Beach, CA 90254',
           '1868 Pacific Coast Hwy Hermosa Beach, CA 90254','200 Pier Ave Suite 221 Hermosa Beach, 90254','200 Pier Ave Suite 126 Hermosa Beach, 90254','1601 Pacific Coast Hwy Hermosa Beach, CA 90254',
           '845 Palm Drive Hermosa Beach, CA 90254','2200 Pacific Coast Hwy Suite 301 Hermosa Beach, CA 90254','2200 Pacific Coast Hwy Suite 316 Hermosa Beach, CA 90254',
           '2200 Pacific Coast Hwy Suite 308 Hermosa Beach, CA 90254','1221 Hermosa Ave Hermosa Beach, CA 90254','2447 Pacific Coast Hwy Hermosa Beach, CA 90254'],
'Lat': [33.864140,33.865050,33.864540,33.869000,33.869070,33.862750,33.862750,33.866680,33.865950,33.870860,33.870860,33.870860,33.862720,33.871640], 
'Lng': [-118.392370,-118.392760,-118.392630,-118.394290,-118.394310,-118.398840,-118.398840,-118.394210,-118.400510,-118.394270,-118.394270,-118.394270,-118.400550,-118.395370],    
'Use': ['Retail','Retail','Retail','Office/Retail','Office/Retail','Office','Office','Office/Retail','Office','Office','Office','Office','Office/Retail','Office'],
'Square Footage': [995,3000,1686,1700,944,306,357,2870,2038,1566,2484,1568,4510,5000],
'Annual Price Per Square Foot': [35.40,42,42,39,39,88.20,69,31.80,42,45,45,45,40.20,45],
'Monthly Lease': [2935.25,10500,5901,5525,3068,2249,2052,7605,7133,5872,9315,5880,15108,18750],
'Year Built': [1983,1941,1987,1970,1970,2008,2008,1988,1998,1982,1982,1982,1923,1985]})

HB_Rental_Data

,Address,Annual Price Per Square Foot,Lat,Lng,Monthly Lease,Square Footage,Use,Year Built
0,"1200 Pacific Coast Hwy Hermosa Beach, CA 90254",35.4,33.86414,-118.39237,2935.25,995,Retail,1983
1,"1402 Pacific Coast Hwy Hermosa Beach, CA 90254",42.0,33.86505,-118.39276,10500.00,3000,Retail,1941
2,"1310 Pacific Coast Hwy Hermosa Beach, CA 90254",42.0,33.86454,-118.39263,5901.00,1686,Retail,1987
3,"1880 Pacific Coast Hwy Hermosa Beach, CA 90254",39.0,33.86900,-118.39429,5525.00,1700,Office/Retail,1970
4,"1868 Pacific Coast Hwy Hermosa Beach, CA 90254",39.0,33.86907,-118.39431,3068.00,944,Office/Retail,1970
5,"200 Pier Ave Suite 221 Hermosa Beach, 90254",88.2,33.86275,-118.39884,2249.00,306,Office,2008
6,"200 Pier Ave Suite 126 Hermosa Beach, 90254",69.0,33.86275,-118.39884,2052.00,357,Office,2008
7,"1601 Pacific Coast Hwy Hermosa Beach, CA 90254",31.8,33.86668,-118.39421,7605.00,2870,Office/Retail,1988
8,"845 Palm Drive Hermosa Beach, CA 90254",42.0,33.86595,-118.40051,7133.00,2038,Office,1998
9,2200 Pacific Coast Hwy Suite 301 Hermosa Beach...,45.0,33.87086,-118.39427,5872.00,1566,Office,1982


#### Clean Retail Listing Data


In [18]:
#Only keep records that can be used for Retail space
#Average size of coffee shop is range from 800-2000 square feet, exclude records where square footage > 2000
Retail = ['Retail','Office/Retail']
HB_Rental_Data = HB_Rental_Data[HB_Rental_Data.Use.isin(Retail)]
HB_Rental_Data = HB_Rental_Data[HB_Rental_Data['Square Footage']<=2000]
HB_Rental_Data

,Address,Annual Price Per Square Foot,Lat,Lng,Monthly Lease,Square Footage,Use,Year Built
0,"1200 Pacific Coast Hwy Hermosa Beach, CA 90254",35.4,33.86414,-118.39237,2935.25,995,Retail,1983
2,"1310 Pacific Coast Hwy Hermosa Beach, CA 90254",42.0,33.86454,-118.39263,5901.00,1686,Retail,1987
3,"1880 Pacific Coast Hwy Hermosa Beach, CA 90254",39.0,33.86900,-118.39429,5525.00,1700,Office/Retail,1970
4,"1868 Pacific Coast Hwy Hermosa Beach, CA 90254",39.0,33.86907,-118.39431,3068.00,944,Office/Retail,1970


#### Add Retail Spaces to Map

In [19]:
coffee_venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Hermosa Beach Pier

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hermosa Beach Pier',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(coffee_venues_map)

# add the coffee shops and cafes as blue circle markers
for lat, lng, label in zip(coffee_shop_df_filtered.lat, coffee_shop_df_filtered.lng, coffee_shop_df_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(coffee_venues_map)
    
# add the coffee related venues (cafes, breakfast, etc) as orange circle markers
for lat, lng, label in zip(other_coffee_df_filtered.lat, other_coffee_df_filtered.lng, other_coffee_df_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        popup=label,
        fill = True,
        fill_color='orange',
        fill_opacity=0.6
    ).add_to(coffee_venues_map)    
    
# add the coffee shops and cafes as blue circle markers
for lat, lng, label in zip(HB_Rental_Data.Lat, HB_Rental_Data.Lng, HB_Rental_Data.Use):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='purple',
        popup=label,
        fill = True,
        fill_color='purple',
        fill_opacity=0.6
    ).add_to(coffee_venues_map)    
    

#display map
coffee_venues_map

#### Now Jane has a map to help her visualize all the coffee shops, coffee related venues, and potential retail space for her own shop all in one place!